In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from CellList import CellList
from VerletList import VerletList
import jax

In [8]:
box_size = 8
positions = jax.random.uniform(jax.random.PRNGKey(0), (int(1e3), 3))*box_size

In [4]:
cell_list = CellList(box = (box_size, box_size, box_size), cutoff = 2.0)
cell_list.get_cell_ids(positions)
idxs = jax.numpy.asarray(list(range(positions.shape[0])))
all_ns = cell_list.get_neighbor_ids(idxs, mask_self=True)

In [5]:
verlet_list = VerletList(cutoff = 2.0)
atom0_n_ids = all_ns[0][all_ns[0] != -1]
neighbors = verlet_list.get_neighborhood(positions[atom0_n_ids, :], positions[0, :])

In [6]:
atom0_real_neighnors = atom0_n_ids[neighbors]

In [7]:
atom0_real_neighnors

Array([993, 988, 968, 961, 959, 951, 947, 923, 891, 872, 871, 862, 828,
       808, 770, 736, 735, 708, 681, 631, 629, 621, 600, 594, 571, 555,
       541, 529, 509, 506, 466, 443, 437, 433, 400, 344, 328, 314, 309,
       283, 250, 242, 197, 145, 119,  98,  95,  47,  40,  22],      dtype=int32)